In [523]:
import pandas as pd
import numpy as np

In [524]:
#import seaborn as sns
#import matplotlib.pyplot as plt
#%matplotlib inline

In [525]:
from datetime import datetime, timedelta
from dateutil import relativedelta

In [526]:
import string

In [527]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.text import Text 
from nltk.collocations import *
from nltk.corpus import wordnet
from nltk import FreqDist

In [528]:
df = pd.read_csv("Smoke_House_Google_Reviews.csv")

In [529]:
df.drop(['query', 'name', 'google_id', 'place_id', 'location_link',
       'reviews_link', 'reviews_per_score', 'reviews','rating','review_id',
       'author_link','author_image','review_img_url',
       'review_link','reviews_id'],axis=1,inplace=True)

In [530]:
df['text_yn'] = df['review_text'].apply(lambda x: 'no' if x!=x else 'yes')
df['anwser_yn'] = df['owner_answer'].apply(lambda x: 'no' if x!=x else 'yes')
print(df['text_yn'].value_counts())
print(df['anwser_yn'].value_counts())

df[df['text_yn']=='yes']['review_text'].isna().value_counts()

df.head()

yes    1143
no      579
Name: text_yn, dtype: int64
no     1180
yes     542
Name: anwser_yn, dtype: int64


,author_title,author_id,review_text,owner_answer,owner_answer_timestamp,owner_answer_timestamp_datetime_utc,review_rating,review_timestamp,review_datetime_utc,review_likes,text_yn,anwser_yn
0,Christian Díaz,118290890328226276251,(Translated by Google) As always a 10 for this...,NaN,NaN,NaN,5,1661161121,08/22/2022 09:38:41,0,yes,no
1,Gabriela Santiana,106821844711863729721,Delicioso todo.. seguramente repetiremos para ...,NaN,NaN,NaN,5,1661152489,08/22/2022 07:14:49,1,yes,no
2,Meir Cohen,104714702665944371857,"Great restaurant, great food, good price, slow...",NaN,NaN,NaN,4,1661120357,08/21/2022 22:19:17,0,yes,no
3,kaka kaka,101906364734052474176,"Está demasiado bueno la calidad es increíble, ...",NaN,NaN,NaN,5,1661114131,08/21/2022 20:35:31,1,yes,no
4,Alberto Martinez,108348994920795296705,(Translated by Google) American food restauran...,NaN,NaN,NaN,5,1661097601,08/21/2022 16:00:01,1,yes,no


In [531]:
rev_text = df[df['text_yn']=='yes']
rev_text = rev_text[rev_text['review_rating'] <= 2]
rev_text = pd.DataFrame(rev_text['review_text'])
rev_text.reset_index(inplace=True)

In [532]:
rev_text.isna().value_counts()

index  review_text
False  False          109
dtype: int64

In [533]:
rev_text.head()

,index,review_text
0,94,(Translated by Google) Incredible food but the...
1,104,(Translated by Google) I've been there twice:\...
2,107,"Coke with no gas, burned steak, disgusting chi..."
3,112,(Translated by Google) We were there yesterday...
4,227,(Translated by Google) bad service\n\n(Origina...


In [534]:
rev_text.iloc[0]

index                                                         94
review_text    (Translated by Google) Incredible food but the...
Name: 0, dtype: object

In [535]:
#rev_text['review_text'] = rev_text['review_text'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
#rev_text.head(3)

In [536]:
#string.punctuation

In [537]:
#stopwords.words('english')

In [538]:
'us' in stopwords.words('english')

False

In [539]:
stopwords.words('english')#.append('us')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [540]:
def text_process(review):
    nopunc = [char for char in review if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    nopunc = [word.lower() for word in nopunc.split()]
    return [word for word in nopunc if word.lower() not in stopwords.words('english')]

In [541]:
rev_text['clean_text'] = rev_text['review_text'].apply(text_process)

In [542]:
#rev_text['text_split'] = rev_text['review_text'].apply(lambda x: str(x).split())

In [543]:
rev_text.head()

,index,review_text,clean_text
0,94,(Translated by Google) Incredible food but the...,"[translated, google, incredible, food, service..."
1,104,(Translated by Google) I've been there twice:\...,"[translated, google, ive, twice, first, took, ..."
2,107,"Coke with no gas, burned steak, disgusting chi...","[coke, gas, burned, steak, disgusting, chips, ..."
3,112,(Translated by Google) We were there yesterday...,"[translated, google, yesterday, wednesday, jul..."
4,227,(Translated by Google) bad service\n\n(Origina...,"[translated, google, bad, service, original, ש..."


In [544]:
def cleaner(split):
    for string in ['translated','google']:
        while string in split:
            split.remove(string)
            
rev_text['clean_text'].apply(cleaner)

0      None
1      None
2      None
3      None
4      None
       ... 
104    None
105    None
106    None
107    None
108    None
Name: clean_text, Length: 109, dtype: object

In [545]:
#rev_text

In [546]:
def cleaner2(review):
    while 'original' in review:
        review.index('original')
        del review[review.index('original'):]
            
rev_text['clean_text'].apply(cleaner2)   

0      None
1      None
2      None
3      None
4      None
       ... 
104    None
105    None
106    None
107    None
108    None
Name: clean_text, Length: 109, dtype: object

In [547]:
rev_text.head(3)

,index,review_text,clean_text
0,94,(Translated by Google) Incredible food but the...,"[incredible, food, service, leaves, lot, desir..."
1,104,(Translated by Google) I've been there twice:\...,"[ive, twice, first, took, wellcooked, santa, f..."
2,107,"Coke with no gas, burned steak, disgusting chi...","[coke, gas, burned, steak, disgusting, chips, ..."


In [548]:
rev_text

,index,review_text,clean_text
0,94,(Translated by Google) Incredible food but the...,"[incredible, food, service, leaves, lot, desir..."
1,104,(Translated by Google) I've been there twice:\...,"[ive, twice, first, took, wellcooked, santa, f..."
2,107,"Coke with no gas, burned steak, disgusting chi...","[coke, gas, burned, steak, disgusting, chips, ..."
3,112,(Translated by Google) We were there yesterday...,"[yesterday, wednesday, july, 20, first, asked,..."
4,227,(Translated by Google) bad service\n\n(Origina...,"[bad, service]"
...,...,...,...
104,1689,"Cold pulled pork, and dry ribs","[cold, pulled, pork, dry, ribs]"
105,1690,"(Translated by Google) Cold, tasteless a pity....","[cold, tasteless, pity, place, nice, also, tak..."
106,1693,They really need to step up their game for it ...,"[really, need, step, game, work, come, back, q..."
107,1706,"(Translated by Google) To not come back, as so...","[come, back, soon, arrive, find, wall, large, ..."


In [549]:
review_text = rev_text['clean_text'].to_list()

In [550]:
all_reviews = []

for review in review_text:
    all_reviews = all_reviews + review

all_reviews

['incredible',
 'food',
 'service',
 'leaves',
 'lot',
 'desired',
 '130',
 'served',
 'food',
 'everything',
 'ordered',
 'took',
 'forever',
 'ive',
 'twice',
 'first',
 'took',
 'wellcooked',
 'santa',
 'fe',
 'arrived',
 'burnt',
 'staff',
 'attentive',
 'customer',
 'spoke',
 'spanish',
 'factor',
 'linguistic',
 'misunderstanding',
 'asking',
 'specifically',
 'foods',
 'without',
 'garlic',
 'difficulty',
 'digesting',
 'deaf',
 'ears',
 'full',
 'oil',
 'chock',
 'full',
 'garlic',
 'second',
 'time',
 'went',
 'exactly',
 'like',
 'first',
 'including',
 'garlic',
 'asking',
 'santa',
 'fe',
 'medium',
 'cooking',
 'obviously',
 'arrived',
 'burnt',
 'basically',
 'would',
 'go',
 'back',
 'staff',
 'seem',
 'annoyed',
 'every',
 'slightest',
 'request',
 'excellent',
 'quality',
 'food',
 'unable',
 'cook',
 'requested',
 'coke',
 'gas',
 'burned',
 'steak',
 'disgusting',
 'chips',
 'bad',
 'quality',
 'meat',
 '70',
 'fat',
 'yesterday',
 'wednesday',
 'july',
 '20',
 'firs

In [551]:
all_reviews_joined = ' '.join(all_reviews)
all_reviews_joined

'incredible food service leaves lot desired 130 served food everything ordered took forever ive twice first took wellcooked santa fe arrived burnt staff attentive customer spoke spanish factor linguistic misunderstanding asking specifically foods without garlic difficulty digesting deaf ears full oil chock full garlic second time went exactly like first including garlic asking santa fe medium cooking obviously arrived burnt basically would go back staff seem annoyed every slightest request excellent quality food unable cook requested coke gas burned steak disgusting chips bad quality meat 70 fat yesterday wednesday july 20 first asked foods people gluten intolerance surprise restaurant name located street many visitors told nooo food crosscontaminated well seemed people restaurant easy clean griddle make hamburger thats little desire work people intolerances allergies food complain cannot allowed need desire gloves clean pan customer leaves happy complicated need like job cheers bad se

In [552]:
nltk_tags = nltk.pos_tag(all_reviews)
nltk_tags

[('incredible', 'JJ'),
 ('food', 'NN'),
 ('service', 'NN'),
 ('leaves', 'VBZ'),
 ('lot', 'RB'),
 ('desired', 'VBN'),
 ('130', 'CD'),
 ('served', 'VBD'),
 ('food', 'NN'),
 ('everything', 'NN'),
 ('ordered', 'VBD'),
 ('took', 'VBD'),
 ('forever', 'RB'),
 ('ive', 'JJ'),
 ('twice', 'NN'),
 ('first', 'RB'),
 ('took', 'VBD'),
 ('wellcooked', 'VBN'),
 ('santa', 'JJ'),
 ('fe', 'NN'),
 ('arrived', 'VBD'),
 ('burnt', 'JJ'),
 ('staff', 'NN'),
 ('attentive', 'JJ'),
 ('customer', 'NN'),
 ('spoke', 'VBD'),
 ('spanish', 'JJ'),
 ('factor', 'NN'),
 ('linguistic', 'JJ'),
 ('misunderstanding', 'NN'),
 ('asking', 'VBG'),
 ('specifically', 'RB'),
 ('foods', 'NNS'),
 ('without', 'IN'),
 ('garlic', 'JJ'),
 ('difficulty', 'NN'),
 ('digesting', 'VBG'),
 ('deaf', 'JJ'),
 ('ears', 'NNS'),
 ('full', 'JJ'),
 ('oil', 'NN'),
 ('chock', 'NN'),
 ('full', 'JJ'),
 ('garlic', 'JJ'),
 ('second', 'JJ'),
 ('time', 'NN'),
 ('went', 'VBD'),
 ('exactly', 'RB'),
 ('like', 'IN'),
 ('first', 'JJ'),
 ('including', 'VBG'),
 ('garli

In [553]:
"""pos_tags = ['J','V','N','R']

for tag in pos_tags:
    pos_filtered = [pos_tuple for pos_tuple in nltk_tags if pos_tuple[1] in pos]
pos_filtered"""

"pos_tags = ['J','V','N','R']\n\nfor tag in pos_tags:\n    pos_filtered = [pos_tuple for pos_tuple in nltk_tags if pos_tuple[1] in pos]\npos_filtered"

In [554]:
def nltk_pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

In [555]:
def lemmatize_reviews(reviews):
    lemmatizer = WordNetLemmatizer() 
    lemmatized_reviews = []
    wordnet_tagged = map(lambda x: (x[0], nltk_pos_tagger(x[1])), nltk_tags)
    for word, tag in wordnet_tagged:
        if tag is None:
            lemmatized_reviews.append(word)
        else:        
            lemmatized_reviews.append(lemmatizer.lemmatize(word, tag))
    return lemmatized_reviews

In [556]:
clean_text_lemmatized = lemmatize_reviews(all_reviews_joined)
clean_text_lemmatized

['incredible',
 'food',
 'service',
 'leave',
 'lot',
 'desire',
 '130',
 'serve',
 'food',
 'everything',
 'order',
 'take',
 'forever',
 'ive',
 'twice',
 'first',
 'take',
 'wellcooked',
 'santa',
 'fe',
 'arrive',
 'burnt',
 'staff',
 'attentive',
 'customer',
 'speak',
 'spanish',
 'factor',
 'linguistic',
 'misunderstanding',
 'ask',
 'specifically',
 'food',
 'without',
 'garlic',
 'difficulty',
 'digest',
 'deaf',
 'ear',
 'full',
 'oil',
 'chock',
 'full',
 'garlic',
 'second',
 'time',
 'go',
 'exactly',
 'like',
 'first',
 'include',
 'garlic',
 'ask',
 'santa',
 'fe',
 'medium',
 'cook',
 'obviously',
 'arrive',
 'burnt',
 'basically',
 'would',
 'go',
 'back',
 'staff',
 'seem',
 'annoy',
 'every',
 'slight',
 'request',
 'excellent',
 'quality',
 'food',
 'unable',
 'cook',
 'request',
 'coke',
 'gas',
 'burn',
 'steak',
 'disgust',
 'chip',
 'bad',
 'quality',
 'meat',
 '70',
 'fat',
 'yesterday',
 'wednesday',
 'july',
 '20',
 'first',
 'ask',
 'food',
 'people',
 'glut

In [557]:
frequency_distribution = FreqDist(clean_text_lemmatized)
frequency_distribution.most_common(20)

[('food', 66),
 ('bad', 39),
 ('us', 32),
 ('meat', 31),
 ('go', 29),
 ('service', 26),
 ('time', 25),
 ('good', 24),
 ('like', 23),
 ('place', 22),
 ('serve', 21),
 ('ask', 21),
 ('cold', 21),
 ('order', 20),
 ('take', 20),
 ('restaurant', 20),
 ('quality', 19),
 ('wait', 19),
 ('dry', 19),
 ('staff', 17)]

In [558]:
clean_text_lemmatized_joined = ' '.join(clean_text_lemmatized)
clean_text_lemmatized_joined

'incredible food service leave lot desire 130 serve food everything order take forever ive twice first take wellcooked santa fe arrive burnt staff attentive customer speak spanish factor linguistic misunderstanding ask specifically food without garlic difficulty digest deaf ear full oil chock full garlic second time go exactly like first include garlic ask santa fe medium cook obviously arrive burnt basically would go back staff seem annoy every slight request excellent quality food unable cook request coke gas burn steak disgust chip bad quality meat 70 fat yesterday wednesday july 20 first ask food people gluten intolerance surprise restaurant name locate street many visitor tell nooo food crosscontaminated well seemed people restaurant easy clean griddle make hamburger thats little desire work people intolerance allergy food complain cannot allow need desire glove clean pan customer leave happy complicate need like job cheer bad service disappointing put hot water us complain tell u

In [559]:
Text(clean_text_lemmatized).concordance('meat')

Displaying 25 of 31 matches:
 burn steak disgust chip bad quality meat 70 fat yesterday wednesday july 20 f
xplain give one star two year ago go meat melt mouth wonderful quantity large 
ke cut everything could make well do meat first time make wait 1 hour food fri
 bad experience serve us piece burnt meat turn deaf ear grievance unprofession
d irony cold starter regular regular meat expensive poor quality three people 
y three people eat three find burger meat compact dry didnt look like black an
order point serve us well do quality meat appreciate wait 1h see serve people 
nacho unfortunately burger wasnt hit meat wasnt juicy bad food couldnt even ea
ry even mess burger make low quality meat texture taste bbq sauce super market
urger absolutely awful steak gad fat meat absolutely bad service i’ve ever ser
 else dry hard chew fry poor quality meat burger beer alright though staff fri
 well since change chef know problem meat horrible even bring green brisket ho
mahawk thought wait go 

In [560]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(Text(clean_text_lemmatized))
sorted(finder.ngram_fd.items(), key=lambda t: (-t[1], t[0]))[:30]

[(('mac', 'cheese'), 8),
 (('bad', 'service'), 7),
 (('come', 'back'), 7),
 (('1', 'hour'), 6),
 (('bring', 'us'), 6),
 (('go', 'back'), 6),
 (('pull', 'pork'), 6),
 (('hour', 'half'), 5),
 (('look', 'like'), 5),
 (('high', 'expectation'), 4),
 (('onion', 'ring'), 4),
 (('plastic', 'cutlery'), 4),
 (('poor', 'quality'), 4),
 (('quality', 'meat'), 4),
 (('serve', 'us'), 4),
 (('taste', 'like'), 4),
 (('tell', 'us'), 4),
 (('20', 'minute'), 3),
 (('bad', 'experience'), 3),
 (('charge', 'us'), 3),
 (('cold', 'food'), 3),
 (('didnt', 'even'), 3),
 (('dont', 'think'), 3),
 (('dry', 'didnt'), 3),
 (('food', 'bad'), 3),
 (('frozen', 'potato'), 3),
 (('high', 'price'), 3),
 (('hour', 'serve'), 3),
 (('long', 'time'), 3),
 (('meat', 'good'), 3)]

In [561]:
trigram_measures = nltk.collocations.TrigramAssocMeasures()
finder = TrigramCollocationFinder.from_words(Text(clean_text_lemmatized))
sorted(finder.ngram_fd.items(), key=lambda t: (-t[1], t[0]))[:30]
#finder.apply_freq_filter(2)
#finder.nbest(trigram_measures.pmi, 20)

[(('wait', '1', 'hour'), 3),
 (('1', 'hour', 'food'), 2),
 (('15', '20', 'minute'), 2),
 (('dry', 'didnt', 'even'), 2),
 (('food', 'bad', 'service'), 2),
 (('give', 'one', 'star'), 2),
 (('go', 'high', 'expectation'), 2),
 (('hour', 'serve', 'us'), 2),
 (('juicy', 'bad', 'food'), 2),
 (('leave', 'without', 'eat'), 2),
 (('non', 'smoke', 'beef'), 2),
 (('rib', 'pull', 'pork'), 2),
 (('serve', 'cold', 'food'), 2),
 (('serve', 'non', 'smoke'), 2),
 (('take', 'long', 'time'), 2),
 (('taste', 'like', 'anything'), 2),
 (('taste', 'like', 'smoke'), 2),
 (('two', 'year', 'ago'), 2),
 (('1', 'hour', 'dish'), 1),
 (('1', 'hour', 'half'), 1),
 (('1', 'hour', 'several'), 1),
 (('1', 'hour', 'wait'), 1),
 (('1', 'star', 'due'), 1),
 (('1', 'star', 'give'), 1),
 (('1', 'table', 'whole'), 1),
 (('10', 'minute', 'final'), 1),
 (('10', 'people', 'return'), 1),
 (('10', 'plate', 'frozen'), 1),
 (('10', 'pm', 'take'), 1),
 (('10', 'time', 'hamburger'), 1)]